In [1]:
%load_ext google.cloud.bigquery

In [2]:
import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/jovyan/keys/bitcoin-graph-212514-7bd6a39573f8.json"

In [3]:
%%bigquery
SELECT * 
FROM `bitcoin-graph-212514.bitcoin.stats_graph_count`

,Label,Nos
0,tx,343504726
1,PRODUCED,934958077
2,address,366152877
3,MINED_IN,343508521
4,block,542598
5,txout,934949568
6,SPENT,878360856
7,LOCKED_AT,783366220


In [ ]:
%%bigquery
SELECT * 
FROM `bitcoin-graph-212514.bitcoin.tx`

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [5]:
from google.cloud import bigquery
client = bigquery.Client()
sql = """
        SELECT * FROM `bitcoin-graph-212514.bitcoin.stats_cluster_distribution`
        """
df = client.query(sql).to_dataframe()

p = figure(
        x_axis_label = "Cluster Size",
        y_axis_label = "Count", 
        y_axis_type = "log", 
        x_axis_type = "log", 
        y_range=[1,140000000], 
        x_range=[1,18000000])

p.circle(df['Size'],df['NoOfclusters'])
show(p)

In [4]:
%%bigquery
SELECT
  b.families,
  a.seed_address AS Initial_Address_Count,
  b.No_Of_Clusters,
  b.size AS Expanded_Address_Count
FROM (
  SELECT
    FORMAT("[%T]",family) families,
    COUNT(*) seed_address
  FROM
    `bitcoin-graph-212514.bitcoin.ransomware_seed_addresses`
  GROUP BY
    family)a
INNER JOIN (
  SELECT
    families,
    COUNT(*) No_Of_Clusters,
    SUM(size) Size
  FROM
    `bitcoin-graph-212514.bitcoin.stats_ransomware_address_expansion`
  WHERE
    families != '["Unknown"]'
  GROUP BY
    families )b
ON
  a.families = b.families
ORDER BY
  b.size DESC
LIMIT
  15

,families,Initial_Address_Count,No_Of_Clusters,Expanded_Address_Count
0,"[""Locky""]",7038,2,7094
1,"[""PopCornTime""]",1,1,1855
2,"[""CryptXXX""]",1,1,1742
3,"[""CryptoLocker""]",2,1,968
4,"[""DMALockerv3""]",9,3,173
5,"[""CryptoTorLocker2015""]",1,1,159
6,"[""Globe""]",8,3,87
7,"[""SamSam""]",44,17,47
8,"[""EDA2""]",2,2,33
9,"[""Flyper""]",2,1,31


## Addresses with most no of transactions

In [9]:

%%bigquery
SELECT
  inputs.input_pubkey_base58 AS input_key,
  COUNT(*)
FROM
  `bigquery-public-data.bitcoin_blockchain.transactions`
JOIN
  UNNEST (inputs) AS inputs
WHERE
  inputs.input_pubkey_base58 IS NOT NULL
GROUP BY
  inputs.input_pubkey_base58
ORDER BY
  COUNT(*) DESC
LIMIT
  1000

,input_key,f0_
0,1NxaBCFQwejSZbQfWcYNwgqML5wWoE3rK4,1899421
1,1dice8EMZmqKvrGE4Qc9bUFf9PX3xaYDp,1600304
2,1LuckyR1fFHEsXYyx5QK4UFzv3PEAepPMK,1204285
3,1dice97ECuByXAvqXpaYzSaQuPVvrtmz6,1105022
4,1P9RQEr2XeE3PEb44ZE35sfZRRW1JHU8qx,832779
5,1J37CY8hcdUXQ1KfBhMCsUVafa8XjDsdCn,751161
6,1dice9wcMu5hLF4g81u8nioL5mmSHTApw,595197
7,,540584
8,1DUb2YYbQA1jjaNYzVXLZ7ZioEhLXtbUru,540015
9,13vHWR3iLsHeYwT42RnuKYNBoVPrKKZgRv,505949


## The incident of a double spent

In [10]:
%%bigquery
SELECT
  *
FROM (
  SELECT
    transaction_id,
    COUNT(transaction_id) AS dup_transaction_count
  FROM
    `bigquery-public-data.bitcoin_blockchain.transactions`
  GROUP BY
    transaction_id)
WHERE
  dup_transaction_count > 1

,transaction_id,dup_transaction_count
0,d5d27987d2a3dfc724e359870c6644b40e497bdc0589a0...,2
1,e3bf3d07d4b0375638d5f1db5255fe07ba2c4cb067cd81...,2
